In [ ]:
import numpy as np
from scipy.fftpack import rfft, irfft
from scipy.signal import blackman

# Use Fast Fourier Transform to decompose signal
# https://en.wikipedia.org/wiki/Fast_Fourier_transform
# http://www-keeler.ch.cam.ac.uk/lectures/understanding/chapter_4.pdf

def initialize(context):

    context.security = sid(16841) # AMZN
    
    # Set commission
    set_commission(commission.PerShare(cost=0.05, min_trade_cost=1.00))

    # set slippage
    set_slippage(slippage.VolumeShareSlippage(volume_limit=0.0025, price_impact=0.01))

def handle_data(context, data):

    # Trade based upon 30-day average price; check curent price and available cash
    N = 30
    prices = np.asarray(history(N, '1d', 'price'))
    
    # Note: Could not resolve issue with deprecated history command; data.history won't execute properly.
    
    # Use Blackman window to contain spectral leakage (Blackman is a "near optional" tapering function that "removes the foot")
    # https://docs.scipy.org/doc/numpy-1.12.0/reference/generated/numpy.blackman.html
    w = blackman(N)
    
    # Calculate FFT of real sequence
    # https://docs.scipy.org/doc/scipy/reference/tutorial/fftpack.html
    y = rfft((np.transpose(prices*w)[0]))
    
    # Zero out low frequencies (rapid changes in signal) (high pass); take inverse of filtered data
    # https://www.ltu.se/cms_fs/1.36192!/file/E0005E_Lecture06_Transforms-version4.pdf
    hf = 5
    y[hf:-hf] = 0
    yinv = irfft(y)

    # Calculate current ratio
    current = np.mean(yinv[:20])
    average = np.mean(yinv) # Average of N elements
    ratio = current/average

    # Trade Rules
    # If current price is 1% above the 10-day average price and there is enough cash, buy
    # If current price is less than average price, close position
    buy_signal = 0.99 # Threshold at which to buy
    sell_signal = 1.0  # Threshold at which to sell

    current_price = data.current(context.security, 'price')
    available_cash = context.portfolio.cash
    
    if ratio < buy_signal and available_cash > current_price:

        # Determine number of shares to buy
        num_shares = int(available_cash/current_price)

        # Buy order
        order(context.security, +num_shares)

    elif ratio > sell_signal:

        # Reset position to zero
        order_target(context.security, 0)

    # Record stock price
    record(stock_price = data.current(context.security, 'price'))